In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import zero_one_loss, accuracy_score
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV
from sklearn.neural_network import MLPClassifier
import wandb

idle_time_data = pd.read_csv('../data/df_points/df_points_18_21_class.csv')

TargetVariable = ['idle_time_class']
Predictors = ['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'dt_start', 'hex_enc', 'start_min', 'month', 'day']

X = idle_time_data[Predictors]
y = idle_time_data[TargetVariable]

PredictorScaler=StandardScaler()
PredictorScalerFit=PredictorScaler.fit(X)
X=PredictorScalerFit.transform(X)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=False)
### shuffle/inorder train and test sets:

sweep_configuration = {
    "project": "MultiLayerPerceptronClassification",
    "name": "MLPC-sweep-scaled-1",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "random",
    "parameters": {
        "hidden_layer_sizes": {
            "values": [8, 16, 32, 64, 128, 256, 512]
        },
        "activation": {
            "values": ['identity', 'logistic', 'tanh', 'relu']
        },
        "solver": {
            "values": ['lbfgs','sgd','adam']
        },
        "alpha": {
            "values": [0.0001,0.0005,0.001, 0.005, 0.01, 0.05]
        },
        "learning_rate": {
            "values": ['constant', 'invscaling', 'adaptive']
        },
        "momentum": {
            "values": [0.9, 0.8, 0.7, 0.6, 0.5]
        }
    }
}


KeyboardInterrupt: 

In [4]:
model = MLPClassifier()
model.fit(X_train,y_train)

/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Unknown label type: (array([-1.34327685, -1.34327685, -1.34327685, ...,  1.33970398,
        0.44537704,  1.33970398]),)

In [3]:
def my_train_func():
    wandb.init()

    _hidden_layer_sizes = wandb.config.hidden_layer_sizes
    _activation = wandb.config.activation
    _solver = wandb.config.solver
    _alpha = wandb.config.alpha
    _learning_rate = wandb.config.learning_rate
    _momentum = wandb.config.momentum

    model = MLPClassifier(hidden_layer_sizes=_hidden_layer_sizes,
                          activation=_activation,
                          solver=_solver,
                          alpha=_alpha,
                          learning_rate=_learning_rate,
                          momentum=_momentum)

    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)

    score_training = model.score(X_train, y_train)
    score_validation = model.score(X_test, y_test)

    acc = accuracy_score(y_test.ravel(), y_pred)
    loss = zero_one_loss(y_test.ravel(), y_pred)

    #wandb.sklearn.plot_feature_importances(model, Predictors)

    wandb.log({"accuracy": acc,"loss": loss})
    wandb.log({"conf_matrix": wandb.plot.confusion_matrix(y_true=y_test, preds=y_pred)})
    #wandb.log({"feature_imp": wandb.sklearn.plot_feature_importances(model, Predictors)})
    wandb.log({"score_training":score_training, "score_validation":score_validation})

# INIT SWEEP
sweep_id_rfc = wandb.sweep(sweep_configuration, project="RandomForestClassifier")
# RUN SWEEP
wandb.agent(sweep_id_rfc, function=my_train_func)

Create sweep with ID: ck8egszh
Sweep URL: https://wandb.ai/jonathanweske/RandomForestClassifier/sweeps/ck8egszh


wandb: Agent Starting Run: dymn54e5 with config:
wandb: 	activation: identity
wandb: 	alpha: 0.0001
wandb: 	hidden_layer_sizes: 256
wandb: 	learning_rate: adaptive
wandb: 	momentum: 0.8
wandb: 	solver: adam


/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


wandb: ERROR Run dymn54e5 errored: ValueError('Unknown label type: (array([-1.34327685, -1.34327685, -1.34327685, ...,  1.33970398,\n        0.44537704,  1.33970398]),)')
wandb: Agent Starting Run: llt9c3mz with config:
wandb: 	activation: tanh
wandb: 	alpha: 0.005
wandb: 	hidden_layer_sizes: 512
wandb: 	learning_rate: constant
wandb: 	momentum: 0.8
wandb: 	solver: sgd
wandb: Ctrl + C detected. Stopping sweep.
